In [196]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urlencode
import urllib.parse
from copy import copy
import requests
import re
import json
import urllib.request

## Scrape event information for a driver

In [222]:
def events_for_a_driver(my_url):
    
    # Load races on url
    raw_html = requests.get(my_url).content
    soup_doc = BeautifulSoup(raw_html, "html.parser")
    events = soup_doc.find_all("div", class_="profile-start-line d-flex justify-content-start w-100 lh-120 py-1 flex-wrap")
    races = []
    
    # Find all WRC races
    events = [e for e in events if e.span.text == ' WRC']

    # List out all informations
    
    for e in events:
        race = {}
        race['Driver'] = soup_doc.find("h4").a.text.strip()
        race['Name'] = e.find("a", title="Show entry info and stats").text
        URL = e.find("a", title="Show entry info and stats")['href']
        race['URL'] = URL
    # Check the URL
        print(URL)
        id1 = re.findall("/entryinfo/(\d+)",URL)[0]
        id2 = re.findall("/entryinfo/.+/(\d+)",URL)[0]
        Year = re.findall("/entryinfo/.+-(\d\d\d\d)/\d+/",URL)[0]
        race['Year'] = Year
        
        
        try:
            rank = e.find("span", class_="fs-09 font-weight-normal text-secondary").text
            ra = re.findall("(\d).",rank)[0]
            race['Race_Result'] = ra
            race['DNG_Reason'] = "Null"
        except:
            ra = e.find("div", class_="profile-start-ret text-danger font-weight-bold")
            race['Race_Result'] = "DNF"
            race['DNG_Reason'] = ra
        
        race['Php_url'] = "https://www.ewrc-results.com/inc/json_driverinfo.php?event="+id1+"&entry="+id2+"&sct=&cat=&lg=en"
        
        print("Php found" + race['Php_url'])
        
        x = urllib.request.urlopen(race['Php_url'])
        raw_data = x.read()
        encoding = x.info().get_content_charset('utf8')  # JSON default
        ssresult = json.loads(raw_data.decode(encoding))
        print(ssresult, len(ssresult['stages'][0][1]))

        
        for idx, pos in enumerate(ssresult['pos'][0][1]):
            print(idx)
            
            stage = race.copy()
            
            stage['p_overall'] = pos      
            stage['stage'] = ssresult['stages'][0][1][idx]
            stage['ssname'] = ssresult['sslist'][idx]
            races.append(stage)
            # print(ssresult['pos'][0][1])
        
        
        print("Php info scraped.")
        
        
    return races



In [223]:
races = events_for_a_driver('https://www.ewrc-results.com/profile/3-sebastien-loeb/')

/entryinfo/73366-eko-acropolis-rally-greece-2022/3424301/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=73366&entry=3424301&sct=&cat=&lg=en
{'pos': [['Overall', [6, 1, 1, 1, 2, 3, 1, 1]]], 'stages': [['Stage results', [6, 1, 1, 1, 8, 3, 1, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16'], 'colors': ['#e2431e', '#f1ca3a']} 8
0
1
2
3
4
5
6
7
Php info scraped.
/entryinfo/73362-safari-rally-kenya-2022/3373957/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=73362&entry=3373957&sct=&cat=&lg=en
{'pos': [['Overall', [5, 3, 3, 5, 14, 22, 28, 16, 13, 12, 12, 11, 10, 10, 10, 10, 9, 8, 8]]], 'stages': [['Stage results', [5, 1, 5, 7, 34, 33, 29, 5, 2, 6, 6, 5, 6, 2, 4, 4, 1, 1, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19'], 'colors': ['#e2431e', '#f1ca3a']} 1

{'pos': [['Overall', [1, 1, 1, 1, 1, 1, 2, 15, 12, 9, 9, 9, 8, 8]]], 'stages': [['Stage results', [1, 3, 1, 4, 4, 1, 5, 85, 3, 1, 9, 2, 1, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15'], 'colors': ['#e2431e', '#f1ca3a']} 14
0
1
2
3
4
5
6
7
8
9
10
11
12
13
Php info scraped.
/entryinfo/6541-rallye-de-france-alsace-2013/397899/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=6541&entry=397899&sct=&cat=&lg=en
{'pos': [['Overall', [7, 3, 2, 1, 1, 2, 4, 4, 5, 4, 3, 4, 4, 4]]], 'stages': [['Stage results', [7, 1, 1, 1, 4, 5, 5, 4, 5, 1, 2, 4, 2, 3]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20'], 'colors': ['#e2431e', '#f1ca3a']} 14
0
1
2
3
4
5
6
7
8
9
10
11
12
13
Php info scraped.
/entryinfo/6535-philips-led-rally-argentina-2013/295714/
Php foundhttps://www.ewrc-results.com/inc/json_d

{'pos': [['Overall', [2, 3, 3, 3, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [2, 3, 2, 1, 1, 2, 2, 3, 1, 8, 8, 3, 1, 1, 2, 3, 1, 4, 5, 2, 1, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS20', 'SS21', 'SS22', 'SS23', 'SS24'], 'colors': ['#e2431e', '#f1ca3a']} 22
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
Php info scraped.
/entryinfo/3488-rally-sweden-2012/139837/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=3488&entry=139837&sct=&cat=&lg=en
{'pos': [['Overall', [4, 6, 5, 4, 4, 3, 11, 8, 7, 7, 7, 7, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6]]], 'stages': [['Stage results', [4, 4, 1, 2, 3, 1, 42, 3, 3, 2, 6, 12, 3, 2, 4, 3, 8, 1, 6, 5, 6, 2, 4, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 

{'pos': [['Overall', [2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 1, 1]]], 'stages': [['Stage results', [2, 2, 2, 1, 1, 2, 3, 1, 1, 2, 1, 2, 2, 1, 3, 2, 3, 1, 1, 1, 2, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22'], 'colors': ['#e2431e', '#f1ca3a']} 22
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
Php info scraped.
/entryinfo/2224-rally-sweden-2011/99867/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=2224&entry=99867&sct=&cat=&lg=en
{'pos': [['Overall', [5, 6, 8, 9, 9, 7, 9, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6]]], 'stages': [['Stage results', [5, 7, 9, 13, 8, 2, 17, 1, 6, 7, 7, 1, 1, 2, 1, 6, 6, 2, 5, 5, 6, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22'], 'colors

{'pos': [['Overall', [4, 1, 1, 1, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]]], 'stages': [['Stage results', [4, 1, 1, 2, 3, 2, 2, 1, 2, 1, 1, 1, 3, 5, 3, 3, 1, 2, 3, 3, 3]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21'], 'colors': ['#e2431e', '#f1ca3a']} 21
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
Php info scraped.
/entryinfo/399-rally-of-great-britain-2009/37667/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=399&entry=37667&sct=&cat=&lg=en
{'pos': [['Overall', [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [1, 1, 1, 2, 1, 2, 2, 1, 1, 2, 1, 1, 2, 5, 1, 4]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16'], 'colors': ['#e2431e', '#f1ca3a']} 16
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
Php info scraped.
/entryinf

...

{'pos': [['Overall', [4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [4, 5, 1, 2, 3, 2, 1, 2, 1, 1, 2, 3, 1, 1, 1, 4, 1, 3, 3, 1, 1, 2, 1, 2, 2, 2, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23', 'SS24', 'SS25', 'SS26', 'SS27'], 'colors': ['#e2431e', '#f1ca3a']} 27
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
Php info scraped.
/entryinfo/10-neste-rally-finland-2006/547/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=10&entry=547&sct=&cat=&lg=en
{'pos': [['Overall', [1, 4, 4, 4, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]]], 'stages': [['Stage results', [1, 4, 4, 2, 1, 1, 2, 1, 2, 2, 1, 8, 2, 2, 2, 3, 3, 3, 2, 2, 3]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS1

{'pos': [['Overall', [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3]]], 'stages': [['Stage results', [2, 2, 1, 1, 2, 6, 1, 1, 1, 1, 1, 1, 2, 3, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15'], 'colors': ['#e2431e', '#f1ca3a']} 15
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
Php info scraped.
/entryinfo/43-omv-adac-rallye-deutschland-2005/1803/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=43&entry=1803&sct=&cat=&lg=en
{'pos': [['Overall', [1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 6, 8, 5, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19'], 'colors': ['#e2431e', '#f1ca3a']} 19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
Php info scraped.
/entryinfo/42-neste-rally-finland-2005/1726/
Php foundhttps

{'pos': [['Overall', [9, 7, 4, 3, 3, 3, 3, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]]], 'stages': [['Stage results', [9, 3, 1, 4, 3, 2, 3, 3, 2, 7, 4, 4, 2, 5, 1, 5, 5, 4, 6]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19'], 'colors': ['#e2431e', '#f1ca3a']} 19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
Php info scraped.
/entryinfo/60-wales-rally-gb-2004/3082/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=60&entry=3082&sct=&cat=&lg=en
{'pos': [['Overall', [2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2]]], 'stages': [['Stage results', [2, 1, 1, 3, 3, 3, 3, 1, 2, 1, 3, 3, 2, 2, 2, 1, 2, 2, 4]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19'], 'colors': ['#e2431e', '#f1ca3a']} 19
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
Php info scraped.
/ent

{'pos': [['Overall', [4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2]]], 'stages': [['Stage results', [4, 1, 1, 1, 4, 5, 3, 1, 3, 5, 2, 4, 5, 2, 5, 1, 6, 7, 4, 13, 11, 8]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22'], 'colors': ['#e2431e', '#f1ca3a']} 22
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
Php info scraped.
/entryinfo/76-tour-de-corse-rallye-de-france-2003/3946/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=76&entry=3946&sct=&cat=&lg=en
{'pos': [['Overall', [7, 3, 2, 1, 1, 1, 1, 18, 17, 16, 15, 14, 14, 14, 13, 13]]], 'stages': [['Stage results', [7, 1, 2, 3, 2, 3, 3, 49, 2, 9, 9, 5, 1, 1, 1, 1]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16'], 'colors': ['#e2431e', '#f1ca3a']} 16
0
1
2
3
4
5
6
7
8
9
10
11
12
13


{'pos': [['Overall', [3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]], 'stages': [['Stage results', [3, 1, 1, 1, 1, 1, 1, 6, 2, 7, 16, 1, 3, 1, 11, 1, 2, 1, 2, 2, 1, 2]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', 'SS19', 'SS20', 'SS21', 'SS22', 'SS23'], 'colors': ['#e2431e', '#f1ca3a']} 22
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
Php info scraped.
/entryinfo/87-neste-rally-finland-2002/5444/
Php foundhttps://www.ewrc-results.com/inc/json_driverinfo.php?event=87&entry=5444&sct=&cat=&lg=en
{'pos': [['Overall', [18, 23, 21, 23, 22, 20, 19, 19, 16, 16, 15, 15, 14, 13, 13, 13, 13, 12, 12, 12, 10, 10]]], 'stages': [['Stage results', [19, 25, 22, 23, 22, 20, 18, 9, 12, 11, 12, 15, 12, 7, 8, 7, 9, 10, 10, 13, 7, 7]]], 'sslist': ['SS1', 'SS2', 'SS3', 'SS4', 'SS5', 'SS6', 'SS7', 'SS8', 'SS9', 'SS10', 'SS11', 'SS12', 'SS13', 'SS14', 'SS15', 'SS16', 'SS17', 'SS18', '

In [224]:
df = pd.DataFrame(races)
df

,Driver,Name,URL,Year,Race_Result,DNG_Reason,Php_url,p_overall,stage,ssname
0,Sébastien Loeb,EKO Acropolis Rally Greece,/entryinfo/73366-eko-acropolis-rally-greece-20...,2022,DNF,[\r\n Alternato...,https://www.ewrc-results.com/inc/json_driverin...,6,6,SS1
1,Sébastien Loeb,EKO Acropolis Rally Greece,/entryinfo/73366-eko-acropolis-rally-greece-20...,2022,DNF,[\r\n Alternato...,https://www.ewrc-results.com/inc/json_driverin...,1,1,SS2
2,Sébastien Loeb,EKO Acropolis Rally Greece,/entryinfo/73366-eko-acropolis-rally-greece-20...,2022,DNF,[\r\n Alternato...,https://www.ewrc-results.com/inc/json_driverin...,1,1,SS3
3,Sébastien Loeb,EKO Acropolis Rally Greece,/entryinfo/73366-eko-acropolis-rally-greece-20...,2022,DNF,[\r\n Alternato...,https://www.ewrc-results.com/inc/json_driverin...,1,1,SS4
4,Sébastien Loeb,EKO Acropolis Rally Greece,/entryinfo/73366-eko-acropolis-rally-greece-20...,2022,DNF,[\r\n Alternato...,https://www.ewrc-results.com/inc/json_driverin...,2,8,SS5
...,...,...,...,...,...,...,...,...,...,...
3094,Sébastien Loeb,Rallye Catalunya - Costa Brava - Rallye de España,/entryinfo/126-rallye-catalunya-costa-brava-ra...,1999,DNF,[\r\n Accident ...,https://www.ewrc-results.com/inc/json_driverin...,33,31,SS3
3095,Sébastien Loeb,Rallye Catalunya - Costa Brava - Rallye de España,/entryinfo/126-rallye-catalunya-costa-brava-ra...,1999,DNF,[\r\n Accident ...,https://www.ewrc-results.com/inc/json_driverin...,32,31,SS4
3096,Sébastien Loeb,Rallye Catalunya - Costa Brava - Rallye de España,/entryinfo/126-rallye-catalunya-costa-brava-ra...,1999,DNF,[\r\n Accident ...,https://www.ewrc-results.com/inc/json_driverin...,32,27,SS5
3097,Sébastien Loeb,Rallye Catalunya - Costa Brava - Rallye de España,/entryinfo/126-rallye-catalunya-costa-brava-ra...,1999,DNF,[\r\n Accident ...,https://www.ewrc-results.com/inc/json_driverin...,31,26,SS6


In [225]:
df.to_csv('Sébastien_Loeb.csv')